# "Digital Gold": A Visual Analysis of Bitcoin as a Hedge Against Inflation and Market Volatility

As inflation creates general unease within our economy, we're seeing a rise in investments into alternative commodities and assets that are said to "hedge"—an investment that's expected to hold or increase its value over time, even as the cost of everything else goes up. One of these assets is Bitcoin ($BTC), a cryptocurrency often dubbed "Digital Gold" based on popular sentiment. 

As governments printed significant amounts of money, especially in recent years, many investors have looked for assets that can protect their wealth from the resulting inflation. Proponents claim Bitcoin, with its fixed supply, is a perfect candidate. This project explores and investigates this popular claim, asking:

- Does Bitcoin's volatility change during periods of high vs. low inflation? A good hedge should ideally be stable, but what if Bitcoin becomes more chaotic and unpredictable precisely when you need it to be a safe haven?
- During major stock market crashes, does Bitcoin act as a 'safe haven' by holding its value, or does it crash even harder?
- Does the relationship between Bitcoin and inflation stay the same over time, or does it change depending on the market environment?

### Mining Metrics: Sourcing Our Datasets
Several key financial and economic datasets are needed to investigate the relationship between Bitcoin, inflation, and broader market behavior. The following data was sourced from Yahoo Finance (via the `yfinance` library) and the Federal Reserve Economic Data (FRED) database.

**1. Bitcoin (BTC-USD)**
> This dataset contains the daily price history of Bitcoin valued in U.S. Dollars, which is central to evaluating its performance as an asset.

- **Source:** Yahoo Finance (`yfinance` Ticker: BTC-USD)
- **Link:** https://finance.yahoo.com/quote/BTC-USD/
- **Features**:
    - **Date:** The trading day
    - **Open/High/Low/Close:** The opening, highest, lowest, and closing prices for the day.
    - **Volume:** The total number of Bitcoins traded.

**2. S&P 500 Index (^GSPC)**
> This data tracks the performance of 500 of the largest publicly-traded companies in the United States, offering a snapshot of the overall health of the U.S. stock market. This is essential for analyzing how Bitcoin behaves during broad market movements and crashes.

- **Source:** Yahoo Finance (`yfinance` Ticker: ^GSPC)
- **Link:** https://finance.yahoo.com/quote/GSPC/
- **Features:** Includes the same trading day, OHLC (Open, High, Low, Close) and Volume data points as the Bitcoin dataset, but for the S&P 500 index.

**3. CBOE Volatility Index (^VIX)**
> Also known as a "fear index," the VIX measures expected market volatility. It is crucial for understanding how Bitcoin's own volatility and price action correlate with periods of market fear and uncertainty.

 - **Source:** Yahoo Finance (`yfinance` Ticker: ^VIX)
 - **Link:** https://finance.yahoo.com/quote/^VIX/
 - **Features:** Contains OHLC data points representing the daily values of the index.

**4. Gold Futures (GC=F)**
> This dataset tracks the price of gold, the traditional safe-haven asset. It provides a direct benchmark to compare against Bitcoin's performance as an inflation hedge and store of value.

 - **Source:** Yahoo Finance (`yfinance` Ticker: GC=F)
 - **Link:** https://finance.yahoo.com/quote/GC=F/
 - **Features:** Contains OHLC and Volume data for gold futures contracts.

**5. U.S Inflation (CPIAUCSL)**
> This is the primary measure of inflation. The dataset tracks the average change in prices paid by urban consumers for a basket of goods and services.

 - **Source:** Federal Reserve Economic Data (FRED Ticker: CPIAUCSL)
 - **Link:** https://fred.stlouisfed.org/series/CPIAUCSL
 - **Features:**
    - **Date:** The date of the observation
    - **Value:** A seasonally adjusted index value (1982-1984 = 100) representing the relative cost of goods.

**6. Effective Federal Funds Rate (DFF)**
> This dataset tracks the interest rate at which commercial banks lend reserves to each other overnight. It reflects the U.S. monetary policy stance, providing essential context on the macroeconomic environment influencing asset prices.

 - **Source:** Federal Reserve Economic Data (FRED Ticker: DFF)
 - **Link:** https://fred.stlouisfed.org/series/DFF
 - **Features:**
    - **Date:** The date of the observation
    - **Value:** The effective federal funds rate, expressed as a percentage.